In [ ]:
# from google.colab import drive # 挂载谷歌云盘
# drive.mount('/content/drive')
# !nvidia-smi # 显示显卡信息
# ''' 符号%代表一直生效，！代表执行完立马结束，不会生效，所以进入目录用% '''
# %cd /content/drive/MyDrive/timeSerise
# ''' 支持的 常用命令1.ls  2.wget  3.gdoint(int(int(int(w))))n  4.mkdir  5.pwd '''
# !ls
# !pip install patool
# !pip install sktime
# !pip install reformer_pytorch

In [3]:
import random
import numpy as np
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
# from exp.exp_imputation import Exp_Imputation
# from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
# from exp.exp_anomaly_detection import Exp_Anomaly_Detection
# from exp.exp_classification import Exp_Classification
# from data_provider.data_creat import *
# import akshare as ak
from torch.utils.tensorboard import SummaryWriter

In [4]:
class Args:
    '''基本配置'''
    # 选项：[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
    task_name = 'long_term_forecast'
    is_training = 1 # 设置为1则进行训练，设置为0则进行测试
    model_id = 'stock_cy'
    des = '20231210' # 实验描述 20231210
    # 模型名称，选项：[Autoformer, Transformer, TimesNet]
    model = 'TimesNet'

    '''数据加载'''
    # 数据集类型,选项：[ETTh1,ETTh2,ETTm1,ETTm2,custom,m4,PSM,MSL,SMAP,SMD,SWAT,UEA]
    data = 'custom'
    test_path = 'raw_stock_cy_301327.csv'
    root_path = './dataset/Stock/'
    # 根据 is_training 的值设置 data_path
    if is_training == 1:
        data_path = 'stock_cy_all_raw.csv'
    else:
        data_path = test_path
    # 预测任务 M:多变量预测多变量, S:单变量预测单变量, MS:多变量预测单变量
    features = 'M'
    # 目标列名，S或MS任务中的目标特征
    target = 'Close'
    # 时间采集粒度，选项：[s:秒, t:分钟, h:小时, d:天, b:工作日, w:周, m:月]
    freq = 'd'
    # 模型检查点的位置
    checkpoints = './checkpoints/'

    '''预测任务'''
    # 输入序列长度,这是用于模型训练的输入序列的长度
    seq_len = 60
    # 开始标记长度,这是模型输出目标中有标签数据的长度，类似于滑动窗口的长度
    label_len = 20
    # 预测序列长度
    pred_len = 20
    # 季节模式（针对M4数据集）
    seasonal_patterns = 'Monthly'
    inverse = False    # 反转输出数据

    '''插补任务'''
    # 插补任务中数据丢失率
    mask_rate = 0.25

    '''异常检测任务'''
    # 异常检测中异常点占比
    anomaly_ratio = 0.25

    '''模型定义'''
    # TimesBlock 中傅里叶变换,频率排名前k个周期
    top_k = 5
    # Inception 中卷积核个数
    num_kernels = 6
    # encoder 输入特征数
    enc_in = 31
    # decoder 输入特征数
    dec_in = 31
    # 输出通道数
    c_out = 31
    # 线性层隐含神经元个数
    d_model = 32
    # FFN 层隐含神经元个数
    d_ff = 32
    # 多头注意力机制
    n_heads = 8
    # encoder 层数
    e_layers = 2
    # decoder 层数
    d_layers = 1
    # 滑动窗口长度
    moving_avg = 20
    # 对 Q 进行采样，对 Q 采样的因子数
    factor = 3
    # 是否下采样操作 pooling
    distil = True
    # dropout 率
    dropout = 0.1
    # 时间特征嵌入方式,选项：[timeF, fixed, learned]
    embed = 'timeF'
    # 激活函数类型
    activation = 'gelu'
    # 是否输出 attention
    output_attention = False

    '''优化'''
    # 测试集的比例
    if is_training == 1:
        test_ratio = [0.1,0.1] # test测试集、valid验证集的占比
    else:
        test_ratio = [0.6,0.1]

    # 并行核心数
    num_workers = 10
    # 实验轮数
    itr = 1
    # 训练迭代次数
    train_epochs = 500
    # batch size 大小
    batch_size = 256
    # early stopping 机制容忍次数
    patience = 5
    # 学习率
    learning_rate = 0.0001
    # 损失函数
    loss = 'MSE'
    # 学习率下降策略
    lradj = 'type1'
    # 使用混合精度训练
    use_amp = False

    '''GPU'''
    # 使用 gpu
    use_gpu = True
    gpu = 0
    # 使用多个 gpus
    use_multi_gpu = False
    # 多 gpu 的设备 id
    devices = '0,1,2,3'

    '''去平稳化投影仪参数'''
    # 投影仪的隐藏层维度（列表）
    p_hidden_dims = [128, 128]
    # 投影仪中的隐藏层数
    p_hidden_layers = 2


# 创建参数对象
args = Args()

# 设置随机种子以确保结果可重现
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


# 获取列数
num_columns = 32
# args.des = NUM
args.enc_in = num_columns - 1
args.dec_in = num_columns - 1
args.c_out = num_columns - 1

In [ ]:
# 检查并设置 GPU
args.use_gpu = torch.cuda.is_available() and args.use_gpu
if args.use_gpu:
    print("使用 GPU.")
    total_cuda_devices = torch.cuda.device_count()  # 获取系统中可用的 GPU 总数
    print(f"系统中总共有 {total_cuda_devices} 个 CUDA 设备可用。")
    if args.use_multi_gpu:
        args.devices = args.devices.replace(' ', '')
        device_ids = args.devices.split(',')
        args.device_ids = [int(id_) for id_ in device_ids]
        args.gpu = args.device_ids[0]

        # 打印多 GPU 使用情况
        print(f"使用多个GPU: {args.device_ids}")
        device = torch.device(f"cuda:{args.gpu}" if args.use_gpu else "cpu")
        print(f"Primary GPU (cuda:{args.gpu}) is in use.")
    else:
        args.gpu = 0
        device = torch.device("cuda" if args.use_gpu else "cpu")
        print("使用单个 GPU.")
else:
    device = torch.device("cpu")
    print("使用 CPU.")

# 选择合适的实验类
if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
# elif args.task_name == 'short_term_forecast':
#     Exp = Exp_Short_Term_Forecast
# elif args.task_name == 'imputation':
#     Exp = Exp_Imputation
# elif args.task_name == 'anomaly_detection':
#     Exp = Exp_Anomaly_Detection
# elif args.task_name == 'classification':
#     Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast  # 默认情况

# 进行训练和测试
if args.is_training:
    # 多次训练模型
    for ii in range(args.itr):
        setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name, # 任务名称
            args.model_id, # 模型id
            args.model, # 模型名称
            args.data, # 数据集名称
            args.features, # 预测任务
            args.seq_len, # 输入序列长度
            args.label_len, # 开始标记长度
            args.pred_len, # 预测序列长度
            args.d_model, # encoder 输入特征数
            args.n_heads, # 多头注意力机制
            args.e_layers, # encoder 层数
            args.d_layers, # decoder 层数
            args.d_ff, # FFN 层隐含神经元个数
            args.factor, # 对 Q 采样的因子数
            args.embed, # 时间特征嵌入方式
            args.distil, # 是否下采样操作 pooling
            args.des,  # 实验描述
            ii) # 实验轮数

        exp = Exp(args) # 创建实验对象
        print('>>>>>>>开始训练 : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting) # 训练模型

        print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting) # 测试模型
        if args.use_gpu:
            torch.cuda.empty_cache()
else:
    # 模型测试
    ii = 0
    setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des,
        ii)
    exp = Exp(args)  # 创建实验对象
    print('>>>>>>>测试 : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting, test=1) # 测试模型
    if args.use_gpu:
        torch.cuda.empty_cache()

使用 GPU.
系统中总共有 1 个 CUDA 设备可用。
使用单个 GPU.
Use GPU: cuda:0
>>>>>>>开始训练 : long_term_forecast_stock_cy_TimesNet_custom_ftM_sl60_ll20_pl20_dm32_nh8_el2_dl1_df32_fc3_ebtimeF_dtTrue_20231210_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 567654
val 70949
test 70947


  5%|▍         | 100/2217 [00:20<08:37,  4.09it/s]

	iters: 100, epoch: 1 | loss: 0.4770142
	speed: 0.2042s/iter; left time: 226352.7837s


  9%|▉         | 200/2217 [00:48<10:41,  3.15it/s]

	iters: 200, epoch: 1 | loss: 0.4202911
	speed: 0.2802s/iter; left time: 310555.1382s


...

 36%|███▌      | 800/2217 [04:19<07:38,  3.09it/s]

	iters: 800, epoch: 18 | loss: 0.3299479
	speed: 0.3232s/iter; left time: 345868.5398s


 41%|████      | 900/2217 [04:51<07:08,  3.08it/s]

	iters: 900, epoch: 18 | loss: 0.3630718
	speed: 0.3235s/iter; left time: 346141.6494s


 45%|████▍     | 990/2217 [05:21<06:39,  3.08it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./tensorboard_logs/